In [1]:
import sys
sys.path.insert(0, '.')
import swarmist as sw

## PSO

In [2]:
pso_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAMETERS (
        C1 = 2.05 BOUNDED BY (0, 8)
        C2 = 2.05 BOUNDED BY (0, 8) 
        CHI = 0.7298 BOUNDED BY (0, 1)
    )
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            VELOCITY= PARAM(CHI) * (
                DELTA 
                + PARAM(C1) * RANDOM() * (BEST-POS)
                + PARAM(C2) * RANDOM() * (SWARM_BEST()-POS)
            )
            POS = POS + VELOCITY
        ) 
    )
)
UNTIL (
    GENERATION = 1000
)
""")

5.017693002301229e-29


## BB

In [3]:
bb_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            MU= (SWARM_BEST()+BEST)/2
            SD = ABS(SWARM_BEST()-BEST)
            POS = RANDOM_NORMAL(LOC=MU, SCALE=SD)
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

4.435994032191429e-32


## FIPS

In [4]:
fips_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAMETERS (
        PHI = 4.1 BOUNDED BY (0, 8)
        CHI = 0.7298 BOUNDED BY (0, 1)
    )
    POPULATION SIZE(40) INIT RANDOM_UNIFORM() WITH TOPOLOGY LBEST SIZE(2)
    SELECT ALL (
        UPDATE (
            NEIGHBORS = NEIGHBORHOOD()
            N = COUNT(NEIGHBORS)
            W = RANDOM(SIZE=N)
            PHI = SUM(W) * (PARAM(PHI) / N)
            PM = AVG(NEIGHBORS, W)
            SCT = PHI * (PM - POS)
            POS = POS + PARAM(CHI) * (DELTA + SCT)
        ) 
    )
)
UNTIL (
    GENERATION = 1000
)
""")

4.694006976506692e-83


## DE (Binomial)

In [5]:
de_bin_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAMETERS (
        F = 0.5 BOUNDED BY (0, 1)
    )
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING BINOMIAL RECOMBINATION WITH PROBABILITY 0.6
        UPDATE (
            POS = PICK_RANDOM(UNIQUE) + PARAM(F) * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE)) 
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

3.6558065849852255e-24


## DE (Exponential)

In [6]:
de_exp_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAMETERS (
        F = 0.5 BOUNDED BY (0, 1)
    )
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING EXPONENTIAL RECOMBINATION WITH PROBABILITY 0.6
        UPDATE (
            POS = PICK_RANDOM(UNIQUE) + PARAM(F) * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE)) 
        ) WHEN IMPROVED = TRUE 
    )
)
UNTIL (
    GENERATION = 1000
)
""")

1.0876740092354786e-11


## DE RAND-TO-BEST

In [7]:
de_r2b_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAMETERS (
        F = 0.5 BOUNDED BY (0, 1)
    )
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING BINOMIAL RECOMBINATION WITH PROBABILITY 0.6
        UPDATE (
            POS = RAND_TO_BEST(WITH PROBABILITY 0.5) + PARAM(F) * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE)) 
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

3.3321742921968114e-05


## DE CURRENT-TO-BEST

In [8]:
de_r2b_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAMETERS (
        F = 0.5 BOUNDED BY (0, 1)
    )
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING BINOMIAL RECOMBINATION WITH PROBABILITY 0.6
        UPDATE (
            POS = CURRENT_TO_BEST(WITH PROBABILITY 0.5) + PARAM(F) * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE)) 
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

0.011838408190218041


## JAYA

In [9]:
jaya_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            ABS_POS = ABS(POS)
            POS = POS + RANDOM() * (SWARM_BEST() - ABS_POS) - RANDOM() * (SWARM_WORST() - ABS_POS)
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

7.51819215818587e-12


## ABC

In [10]:
abc_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        USING RANDOM RECOMBINATION SIZE(1)
        UPDATE (
            POS = RANDOM_UNIFORM(LOW=-1, HIGH=1) * (POS - PICK_RANDOM())
        ) WHEN IMPROVED = TRUE
    )
    SELECT SIZE(1) WHERE TRIALS > POPULATION_SIZE*NDIMS ORDER BY TRIALS DESC (
        INIT RANDOM_UNIFORM()
    )
)
UNTIL (
    GENERATION = 1000
)
""")

1.682155356483376e-33


## CS

In [11]:
cs_res = sw.sdl.execute("""
SEARCH(
    VAR X SIZE(20) BOUNDED BY (-5.12, 5.12) 
    MINIMIZE SUM(X**2)
)
USING (
    PARAMETERS (
        PA = 0.25 BOUNDED BY (0, 1)
        ALPHA = 1 BOUNDED BY (0, 1)
        MU = 1.5 BOUNDED BY (0,3)
    )
    POPULATION SIZE(40) INIT RANDOM_UNIFORM()
    SELECT ALL (
        UPDATE (
            POS = POS + PARAM(ALPHA) * RANDOM_LEVY(LOC=PARAM(MU)) * (POS - SWARM_BEST())
        ) WHEN IMPROVED = TRUE
    )
    SELECT ALL (
        USING RECOMBINATION WITH PROBABILITY PARAM(PA)
        UPDATE (
            POS = POS + RANDOM() * (PICK_RANDOM(UNIQUE) - PICK_RANDOM(UNIQUE))
        ) WHEN IMPROVED = TRUE
    )
)
UNTIL (
    GENERATION = 1000
)
""")

1.4699860809205262e-05
